In [5]:
# Importando bibliotecas necessárias
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import numpy as np

# URL do dataset Wine
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data'

# Nome das colunas em português BR
column_names_pt_br = [
    'classe',
    'alcool',
    'acido_malico',
    'cinzas',
    'alcalinidade_de_cinzas',
    'magnesio',
    'fenois_totais',
    'flavanoides',
    'fenois_nao_flavanoides',
    'proantocianinas',
    'intensidade_de_cor',
    'matiz',
    'od280_od315_de_vinhos_diluidos',
    'prolina'
]

# Ler o arquivo CSV com as colunas especificadas, definindo a coluna 'classe' como object
vinhos = pd.read_csv(url, names=column_names_pt_br, dtype={'classe': object})

# Separação de X e y
X = vinhos.iloc[:, 1:]  # Variáveis preditoras
y = vinhos['classe']    # Variável-alvo

# Função para executar o experimento com k-fold cross-validation
def executar_experimento(random_state, ks, n_splits=10):
    resultados = []
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)

    for k in ks:
        acuracias = []
        for train_index, test_index in kf.split(X):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]

            # Treinando o modelo KNN
            knn = KNeighborsClassifier(n_neighbors=k)
            knn.fit(X_train, y_train)

            # Previsões e cálculo da acurácia
            y_pred = knn.predict(X_test)
            acuracia = accuracy_score(y_test, y_pred)
            acuracias.append(acuracia)

        # Cálculo da média e do desvio padrão das acurácias
        media = np.mean(acuracias)
        desvio_padrao = np.std(acuracias)
        resultados.append({'random_state': random_state, 'k': k, 'media': media, 'desvio_padrao': desvio_padrao})

    return resultados

# Parâmetros do experimento
random_states = [42, 17, 24]
ks = [3, 5]

# Executando os experimentos
todos_resultados = []
for random_state in random_states:
    resultados = executar_experimento(random_state, ks)
    todos_resultados.extend(resultados)

# Convertendo os resultados para um DataFrame
df_resultados = pd.DataFrame(todos_resultados)

# Exibindo os resultados
print("Resultados do Experimento:")
print(df_resultados)

# Análise dos melhores resultados
print("\nMelhor Configuração para cada Random State:")
for random_state in random_states:
    sub_df = df_resultados[df_resultados['random_state'] == random_state]
    melhor_k = sub_df.loc[sub_df['media'].idxmax()]
    print(f"Random State {random_state}: k={melhor_k['k']}, Média de Acurácia={melhor_k['media']:.4f}, Desvio Padrão={melhor_k['desvio_padrao']:.4f}")

# Análise final de empates
print("\nAnálise de Empates:")
for random_state in random_states:
    sub_df = df_resultados[df_resultados['random_state'] == random_state]
    if len(sub_df[sub_df['media'] == sub_df['media'].max()]) > 1:
        print(f"Empate detectado no Random State {random_state} entre os valores de k={list(sub_df[sub_df['media'] == sub_df['media'].max()]['k'])}")

Resultados do Experimento:
   random_state  k     media  desvio_padrao
0            42  3  0.703595       0.139187
1            42  5  0.664052       0.100075
2            17  3  0.698693       0.104030
3            17  5  0.687582       0.112118
4            24  3  0.712745       0.096816
5            24  5  0.712745       0.093973

Melhor Configuração para cada Random State:
Random State 42: k=3.0, Média de Acurácia=0.7036, Desvio Padrão=0.1392
Random State 17: k=3.0, Média de Acurácia=0.6987, Desvio Padrão=0.1040
Random State 24: k=3.0, Média de Acurácia=0.7127, Desvio Padrão=0.0968

Análise de Empates:
Empate detectado no Random State 24 entre os valores de k=[3, 5]
